 # 一個聲部的tracks搭配其和弦進行及節拍以產生旋律(音高)

In [7]:
from decimal import *
from random import randint

chord = {
    'C':(0, 4, 7),
    'Cmaj7':(0, 4, 7, 11),
    'Dm':(2, 5, 9),
    'Dm7':(2, 5, 9, 12),
    'Em':(4, 7, 11),
    'Em7':(4, 7, 11, 14),
    'F':(5, 9, 12),
    'Fmaj7':(5, 9, 12, 16),
    'G':(7, 11, 14),
    'G7':(7, 11, 14, 17),
    'Am':(9, 12, 16),
    'Am7':(9, 12, 16, 19),
    'Bb':(10, 14, 17),
    'Bdim':(11, 14, 17)
}

In [31]:
def varChord(curChord):
    randomInt = randint(0,5) #不變化 if randomInt == 0
    notes = ''
    for i, component in enumerate(chord[curChord], 1):
        #根音高八度(第一轉位) or #根音及三度音高八度(第二轉位) or #全部高八度
        if (randomInt == 1 and i == 1) or (randomInt == 2 and (i == 1 or i == 2)) or randomInt == 3:
            component += 12
        #全部低八度 or #如為三和弦，五度音低八度(第二轉位)；如為七和弦，七度音低八度(第三轉位)
        elif randomInt == 4 or (randomInt == 5 and ((len(chord[curChord]) == 3 and i == 3) or (len(chord[curChord]) == 4 and i == 4))):
            component -= 12
        component += 60
        notes += str(component) + ','
    return notes[:-1] + ';'

In [32]:
def nextChord(beatSum, indexAcc):
    indexAcc += 1
    curChord = chordProg[indexAcc]
    diff = beatSum - Decimal(0.5)
    if diff >= 0.5:
        beatSum, indexAcc, curChord = nextChord(diff, indexAcc)
    else:
        beatSum = diff
    return beatSum, indexAcc, curChord

In [33]:
def accompanyMelody(staff, chordProg):
    staffReturn = {}
    for key in sorted(staff.keys()):
        indexAcc = 0
        curChord = chordProg[indexAcc]
        beatSum = 0
        re = ''
        totalBeatSum = 0
        for beat in staff[key].split(';'):
            duration = beat.split(',')[0]
            noteSign = beat.split(',')[1]

            #Determine when to select next chord
            if beatSum >= 0.5:
                beatSum, indexAcc, curChord = nextChord(beatSum, indexAcc)

            #Generate different strings according to if it's a note or a rest
            if noteSign == '1' and curChord != '0':
                re += duration + ',' + varChord(curChord)
            else:
                re += duration + ',0;'

            #Add 0.0000000000001 to beatSum while it's a tuplet note
            if Decimal(duration) * 128 % 1 == 0:
                beatSum += Decimal(duration)
            else:
                beatSum += Decimal(duration) + Decimal(0.000000001)

        staffReturn[key] = re[:-1]
    return staffReturn

In [36]:
chordProg = ['C','G','0','0','F','Cmaj7','Dm','G7','C','G','Am','Em','F','Cmaj7','Dm','G7']

staff = {
    '100':'1,1;0.5,1;0.5,1;1,1;0.5,1;0.5,1;1,1;0.5,1;0.5,1;0.125,1;0.125,1;0.125,1;0.125,1;0.125,1;0.125,1;0.125,1;0.125,1;0.125,1;0.125,1;0.125,1;0.125,1;0.125,1;0.125,1;0.125,1;0.125,1',
    '101':'0.4895833333333333333333333333,0;0.01041666666666666666666666667,1;0.5,1;0.25,1;0.125,1;0.125,1;0.125,1;0.25,1;0.125,1;0.25,1;0.0625,1;0.0625,1;0.0625,1;0.0625,1;0.125,1;0.125,1;0.125,1;0.125,1;0.625,0;0.375,1;0.4895833333333333333333333333,0;0.01041666666666666666666666667,1;0.5,1;0.25,1;0.125,1;0.125,1;0.125,1;0.25,1;0.125,1;0.25,1;0.0625,1;0.0625,1;0.0625,1;0.0625,1;0.125,1;0.125,1;0.125,1;0.125,1;0.625,0;0.375,1'
}

test = accompanyMelody(staff, chordProg)

for key in test:
    for element in test[key].split(';'):
        print element
    print
print test

1,72,64,67
0.5,0
0.5,0
1,77,81,84
0.5,74,65,69
0.5,79,83,86,89
1,60,64,67
0.5,69,72,76
0.5,76,67,71
0.125,77,69,72
0.125,65,69,60
0.125,53,57,60
0.125,65,69,72
0.125,60,64,67,71
0.125,60,64,67,71
0.125,60,64,67,59
0.125,60,64,67,71
0.125,74,65,69
0.125,62,65,69
0.125,74,77,81
0.125,62,65,69
0.125,55,59,62,65
0.125,67,71,74,65
0.125,79,83,86,89
0.125,55,59,62,65

0.4895833333333333333333333333,0
0.01041666666666666666666666667,60,64,67
0.5,79,71,74
0.25,0
0.125,0
0.125,0
0.125,0
0.25,0
0.125,0
0.25,65,69,60
0.0625,65,69,60
0.0625,65,69,72
0.0625,65,69,60
0.0625,77,81,84
0.125,72,76,79,83
0.125,48,52,55,59
0.125,72,76,67,71
0.125,60,64,67,71
0.625,0
0.375,67,71,74,77
0.4895833333333333333333333333,0
0.01041666666666666666666666667,60,64,55
0.5,79,71,74
0.25,69,72,76
0.125,69,72,64
0.125,81,72,76
0.125,64,67,59
0.25,52,55,59
0.125,52,55,59
0.25,65,69,60
0.0625,53,57,60
0.0625,65,69,60
0.0625,77,69,72
0.0625,65,69,72
0.125,48,52,55,59
0.125,60,64,67,59
0.125,60,64,67,71
0.125,60,64,67,59
0